<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/word_and_countryfinding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
cd /content/drive/My Drive/AI/basic/data/names_train

/content/drive/My Drive/AI/basic/data/names_train


In [35]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


# from name_dataset import NameDataset
# from torch.nn.utils.rnn import pack_padded_sequence, pad_paked_sequence

In [36]:
# xy1 = pd.read_csv("names_test.csv") #q is it require to convert it(text) into torch
# xy1.shape
# name = (xy1.iloc[0,:][0])
# name

Tast1 : create database

In [62]:
class Name(Dataset):
  def __init__(self, csv_file, root_path, transform=None):
  # def __init__(self, csv_file):
    self.xy = pd.read_csv(csv_file)
    self.root_dir = root_path
    self.transform = transform
    
  def __getitem__(self, id):
    self.word = self.xy.iloc[id,:][0] #iloc will tell all element of row ( ie id) then last [no ie 0] is first element
    self.country = self.xy.iloc[id,:][1]
    # word1 = torch(word)
    # conuntry1 = torch(country)
    return self.word, self.country

  def __len__(self):
    self.len = self.xy.shape[0]
    return self.len

tast2: Create dataloader

In [67]:
# train loader
BS = 1
# database = 
database = Name('/content/drive/My Drive/AI/basic/data/names_train/names_train.csv','/content/drive/My Drive/AI/basic/data/names_train' )
train_loader = DataLoader(dataset=database, batch_size= BS, shuffle=True)
# test_loader = 
print(len(database))
print(database[2000])

13373
('Samaha', 'Arabic')


In [65]:
#test loader
if __name__=='__main__':
  database = Name('/content/drive/My Drive/AI/basic/data/names_train/names_test.csv','/content/drive/My Drive/AI/basic/data/names_train' )
  # database = Name('names_test.csv')
  print(len(database))
  print(database[2000])

6699
('Bakshanski', 'Russian')


task3: model: embedding, rnn, linear layer 
task4: optimizer
task5: training
taask6: test

In [56]:
class net(torch.nn.Module):
  def __init__(self):
    super(net,self).__init__()
    self.word_len = 18 # seq chenged from 15 to 18 for test
    self.vocab_size = 6699 # no of words
    self.embedding_dim =6 # how many diamention it wants
    self.input_size = self.embedding_dim #6
    self.hiddenO = self.input_size #6 dim
    self.hidden_size = self.embedding_dim #6
    self.batch = 1
    self.num_layer =1
    self.num_class = 18 #num of contries

    #embedding
    self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embedding_dim)
    #gru    
    self.grucell = nn.GRU(input_size = self.input_size, hidden_size = self.hidden_size, num_layers =self.num_layer, batch_first = True)
    #fc
    self.fc = nn.Linear(in_features= self.embedding_dim, out_features= self.num_class)
  
  def forward(self, word, contry):
    # word, contry = data
    #converting work in to integer ASCII for input and outpuut ord
    word_ascii, w_len = asc(word, self.word_len) #in: word * BS and out: ASCII * BS with padding
    # challenge we need some padding. keep every word as 10 character long.
    
    word_embed = self.embed(word_ascii) #embedding e extra in matrix out : BS* word_len*E
    #rnn in and out rnn_in BS*Seq*inu_D(e) and rnn_out BS*Seq*hiddenO
    # input = B*Seq_len*input, hidden = num_lauyer* batch,hidden_size
    input = word_embed # B*S*I
    hidden = self.hidden_init()
    out, h_n = self.grucell(input, hidden) #out B*S*hidden [1 10 6] , h_n = num_layerxBxHidden [1 1 6]

    #FC linear layer in Seq to out no of classor contry(-1, nod of class or country)
    out = self.fc(h_n) #in [1 1 6] out [1 1 18]
    out = out.view(self.num_class, -1)
    # out - out.t()
    country, counry_len = asc(contry, self.word_len) # 18 contoury with each 15 word
    country = country.t()#view(-1)
    print("out shaepe", out.shape, "countey shape", country.shape)
    # country = country.squeeze_()
    
    # out = out.squeeze_()
    return out, country

    #output output of DC 
  def hidden_init(self):
    hid = torch.zeros(self.num_layer, self.batch, self.hiddenO)
    return 

  def asc(self, words, word_len):
    word_len = word_len

    for word in words:
      # for cha in word:
      sorks = [ord(cha) for cha in word]
      sorks2 = sorks + [0]*(word_len-len(sorks)) 
      sorks3 = Variable(torch.LongTensor([sorks2]))
    return sorks3, len(sorks2) 

  # sorks= []
  # i=0
  # def asc(self, words, word_len):
  #   word_len = word_len

  #   for word in words:
  #     # for cha in word:
  #     sorks = [ord(cha) for cha in word]
  #     sorks2 = sorks + [0]*(word_len-len(sorks)) 
  #     sorks3 = Variable(torch.LongTensor([sorks2]))
    # return sorks3, len(sorks2)
model = net()

Process contry differently as each contry will have unique no (one number not as matrix as word (N * no of class) will be comparing or predicting the perticular class. so its a one idea to keep perticular class as one number rather than keeping it as one class as array.

TODO each contry as one unique no..

In [57]:
Criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .001)


Trainig

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss

In [58]:
for epoch in range(5):
  for batch_id, (word, contry) in enumerate(train_loader):
    out, country = model(word, contry)
    # print("out shape", out[0].shape, "countey[1] shape", country[1].shape)
    print("out shape", out.shape, "countey[1] shape", country.view(-1).shape)
    optimizer.zero_grad()
    loss = Criterion(out, country)
    loss.backword()
    optimizer.step()
    if batch_idx % 500  == 0:
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


out shaepe torch.Size([18, 1]) countey shape torch.Size([18, 1])
out shape torch.Size([18, 1]) countey[1] shape torch.Size([18])


RuntimeError: ignored

In [ ]:
n= ["name"]
# n[1]
# ord(n[1])
print(asc(n,10)[0].size())
a, b = asc(n,10)
embed=nn.Embedding(200,3)
emb2 = embed(a)



In [ ]:
print(emb2.shape)

In [ ]:
  sorks= []
  i=0
  def asc(words, word_len):
    word_len = word_len

    for word in words:
      # for cha in word:
      sorks = [ord(cha) for cha in word]
      sorks2 = sorks + [0]*(word_len-len(sorks)) 
      sorks3 = Variable(torch.LongTensor([sorks2]))
    return sorks3, len(sorks2)

In [ ]:
n[1]

In [ ]:
n1 =asc(n)

In [ ]:
n2 = concat(n1, zeros[2])

In [ ]:
like_zero[2]